# Datatsets  

To be used for training, validation, and test data. They need to be the cleanest of the clean.  

I have a table of them that I will upload also, I just don't love the tables in markdown (and it's a fairly lopsided, ugly table anyway). To be uploaded when I find all the papers I need to reference.  


## Phyla  

#### Porifera

Sponges, as usual, are difficult to come by. Even more so since Compagen doesn't exist anymore.  

I looked for Sycon, because I know it's supposed to be one of the better ones, but since they took the Compagen site down, it is harder to find. I think I did at this link: https://datadryad.org/stash/dataset/doi:10.5061/dryad.tn0f3. But if I can't find the reads (and it doesn't look like they were ever posted anywhere) it won't be much good. So there's that.  

Right now I have these:  
1. Amphimedon queenslandica  
2. Ephydatia meulleri  


#### Ctenophora  

Ctenos are also pretty scarce. I contacted Joe to get some reads for Mnemiopsis, and he has some others as well, so I'll get what I can from there. 

Current ctenos:  
1. Mnemiopsis leidyi  
2. Pleurobrachia bachei
3. Beroe  

#### Cnidaria  

Definitely more cnidos to be had, so that's good. I discovered that the best way to search for datasets appears to be looking up genome assemblies on NCBI (if you go to the legacy genome assembly page, there is a place over on the right that says "FTP directory for GenBank assembly" (or it might say RefSeq sometimes) and if you click on that, it will take you to a page that looks like it was made in 1997 that is really just a directory full of resources. Find the one that ends in "genomic.fnz.gz" and right click to save the link. That's the only way I know to get the URL you need to curl.) and then looking up the raw read sets on ENA.  

I prioritized chromosome-level assemblies, but also took some scaffold ones. Also didn't pay too much attention to taxonomy, so closely related things were okay, and not having some groups represented was okay. I'm trying to get a good volume of reads, and I don't think it matters a whole lot that some of them come from the same genus.  

Current cnidos:  
1. Acropora hyacinthus  
2. Acropora millepora  
3. Cassiopea andromeda  
4. Aurelia aurita  
5. Diadumene lineata  

#### Mollusca  

For these guys, there seemed to be more species available, but fewer reads for each species. Hopefully it won't make a huge difference.  

Current mollusks:  
1. Achatina immaculata  
2. Archivesica marissinica  
3. Arion vulgaris  
4. Conus ventricosus  
5. Crassostrea gigas  
6. Dreissena polymorpha  
7. Gigantopelta aegis  
8. Lanistes nyassanus  
9. Margaritifera margaritifera  
10. Mercenaria mercenaria  
11. Pomacea canaliculata  
12. Ruditapes philippinarum  

## Processing  

These are the reads I'm going to start with for now. One of the best methods I know to get decontaminated reads is to map them onto a good assembly, so that's the goal for these sets. But before I do that, there are lots and lots of processing steps to take care of, so let's get to it.  

Most of this (minus naming basically) is happening from outside the directory that houses the actual data, here: `/scratch/genomics/spillanej/contamination`.  As I get more and more files, I will have to move things around so that I don't end up hitting my quota and cancelling all my jobs. Rebecca lined up some more space (5TB, unscrubbed) for me, here: `/store/public/spillanej`. I want to think about how I'll use it, but I'm okay for now.  

### Getting the reads and assemblies to a usable place  

### Genomes  

Genomes don't take up nearly the space that reads to, but I still want to be mindful of how many copies of them I have.  

Process:  
1. Download all genome files  
2. Rename to match Genus_species.fasta  
3. Format fastas to get rid of non-nucleotides  
4. Make the header names simple and informative(?)  
5. Index each genome for mapping  

#### Renaming

The assemblies all have names that are super similar. This is annoying. I have been renaming them as they download so that I don't get confused and end up with a bunch of incorrectly named genome assemblies. That would be a disaster. I have also been renaming the read files, either as soon as they download, or when I've downloaded all the ones I'm using from a particular platform and am catting them (or zcatting them) together.  

#### Formatting fastas  

Remember how the Actinia genome had some weird non-nucleotides in it? Well I do. And anvi'o had a script that solved that problem. Presumably other programs do too, but I already installed anvi'o and know how to use that script, so you can guess which program I decided to use for this. Am I sure that they all need this? No, definitely not. Some might be totally fine. But in the interest of treating them all the same way, I'm going to run it for all of them. It certainly won't hurt them if they are already good to go.  

`anvi-script-reformat-fasta datasets/Actinia_tenebrosa.fasta -o formatted_genomes/Actinia_tenebrosa.formatted.fasta --seq-type NT`  

This script also technically can change the headers so that they are simplified, but it doesn't seem to allow for much customization, so they don't end up very informative. As in, they would all be labeled the exact same way, so if I ever needed to identify what species we were talking about, it might be difficult. Would this kind of simplifying be fine for my purposes? Maybe, but since I'm not sure, I'm not going to risk it. See below for what I actually did.  

#### Simplifying the headers  

I don't know if I need to care about the read headers, but I'm hoping not. The assembly headers might not be super important after the mapping stage, but I still feel the need to fix them before I do that step, just in case. I guess working with Anvi'o has made me paranoid. So I used my tried-and-true but fairly janky way of making the headers in the genome assemblies simple and informative. From inside the directory that houses these (formatted_genomes), this is what this looks like:  

`awk '/^>/{print ">Genus_species_" ++i; next}{print}' Genus_species.formatted.fasta > ../simple_headers/Genus_species.fa`  

### Reads  

I want to treat all the read files the same too (according to platform) so even though they might not all need all the steps, I'm still going to do them for all files.  

1. Download read files  
2. Unzip read files  
3. Cat read files together according to platform  
4. Trim all reads (aggressive adapter trimming and nonexistent quality trimming)  
5. Map them to their respective genomes  
6. Save all those that map in new fastq files  

I'm going to have to see how large the mapped read files are, but my thought is that I'll be able to keep them in my /scratch/genomics space, and can transfer the trimmed read files (before mapping) and the bam files to my /store/public space. Zipped, of course. The original read files I'll delete, as I can just download them again if I ever need them again.  

#### Downloading   

I'm just using curl commands to get them all downloaded. Before, I was doing this on the login node, which is fine, but moving forward I'm going to pop these commands into scripts that I can submit as jobs.  

#### Samtools steps  

I was trying to figure out what samtools processing steps I need to go through, since these files take up a lot of space and I don't want to make ones that I don't really need. But I definitely need a bam file of all the reads that mapped to the genome, and I need the sorted version for blobtools to work.  

Rebecca mentioned that she has not had good luck with the filtering of these files. That sometimes the file sizes between the files at different stages of filtering are the same size. So I am doing a trial run on one of them (Acropora hyacinthus). I'm having it spit out another stage of sam file, one from after it should have filtered out all of the things that don't map. Hopefully they are different, and I can look at it enough to feel confident about the samtools filtering step.

### Snakemake  

I'm also working on learning Snakemake (here's some docs: https://snakemake.readthedocs.io/en/stable/index.html), so I'm hoping to do most of these things from in Snakemakes platform/format/style/syntax. I did one batch of genomes first, and got them as far as the genome indexing step, which was the first step I figured out how to do in Snakemake. I will definitely upload my Snakefile to github, but I'll also probably drop pieces of it in here as I make rules.  

### Space!  

I knew this was coming, but I have officially run out of space in my scratch/genomics directory! Good thing Rebecca made that other one for me. I'm going to transfer all of the raw reads there, at least until I get them trimmed. Then I will probably delete all the raw ones, and just keep the trimmed ones. I have all of the accessions recorded, so if I really need to get some or all of them back at some point, I can.  

I'm going to have to be a little strategic about this. The compute nodes of Hydra can't really interact with files in the /store/public/spillanej/ space, so I need to use it for storage and not as a place to keep things I'm interacting with more often. That, combined with the fact that these files take a while to move around means I'd rather keep them somewhere slightly more accessible, so I'm going to strategize what the best thing to do with my storage space is. I want it to be useful, and to keep things that I don't need right this minute, and that will free up some space to work in my scratch directories.  

ALSO, I found out that the way I catted (or rather, zcatted) the reads together is freaking out the process, so I'm going to have to redownload the ones I did that to. Everything in the first batch (Ephydatia muelleri, Pleurobrachia bachei, Nematostella vectensis, Cassiopea andromeda, Acropora hyacinthus, Acropora millepora, Actinia tenebrosa, Amphimedon queenslandica, Aurelia aurita) should be fine, since I unzipped those before catting them together, but all the ones I did second (Diadumene lineata (illumina reads only), Arion vulgaris, Crassostrea gigas, Dreissena polymorpha (pacbio reads only), Lanistes nyassanus, Margaritifera margaritifera, Mercenaria mercenaria, Pomacea canaliculata (illumina reads only), Ruditapes philippinarum) will need to be redownloaded. I actually don't have to do this for all of them, because if the read files were singular, I never catted them, and they are still fine (Diadumene lineata (pacbio reads only), Achatina immaculata, Archivesica marissinica, Conus ventricosus, Dreissena polymorpha (illumina reads only), Gigantopelta aegis, Pomacea canaliculata (pacbio reads only)). This is a bummer, because that took a while, but it will be an opportunity to automate more of the process, and it will help free up space in the short term. 

**Oof, this space thing is really coming for me**  
I keep having jobs that don't finish correctly because I'm hitting my quota. I didn't realize, but the /pool/genomics/spillanej/ space is different from my current space, so I'm going to start writing a ton of files there. And that's where I'll transfer all the sorted bam files and all of the reads that have been mapped to the genomes. This should give me much more breathing room than before.  

**Ok, but not really**  
It turns out there are only 2Tb of space in the /pool/genomics spot, so not as much writing there! But I did get my quota temporarily expanded in the /scratch/genomics spot, so no more worries about space for a bit.

As I finish doing the mapping for each dataset, I've been managing the space all the things are taking up. Since I can't access the /store/public/spillanej/ space from compute nodes, I've been storing things (logical use, I know) that I won't need now but might be useful to have later. I've put all of the results from the decontamination tests I ran earlier there, along with all the sam files for all the marine invert datasets and the trimmed reads (the ones before mapping). I didn't save raw reads because I can always download those again later from the repositories if I need them, but I chose the trimmed ones because they have already been processed a bit. And the sam files can be converted into any kind of related file that I need, so I saved those too.  

I ended up running out of space in the store space, so I stashed things I did not have room for in the /pool/genomics/spillanej/ space in the corresponding directories. I think this is just true for the Amphimedon trimmed reads right now.  

I am also going to move all of the sorted bam files over to the pool/genomics space. I'm pretty sure there is room, and I can access that from compute nodes, so when I want to do something with them, they will just all live in one spot.

Okay, so as things stand on 7-8-22, all of the sorted bams are in the pool/genomics space, all of the mapped reads are in the scratch/genomics space. All of the trimmed reads are in the store/public space, along with all of the decontamination test results that would fit. I'll keep the test results there for now, but I don't think I'm committed to them long term.  

### Further Decontamination  

The decontamination process is not over just because I've mapped all the reads to their genomes! There could still be lurking contaminants. I think I'm going to do this decon step in two different ways: one where I very carefully try to pick out each little tiny contaminant read, and one where I take a quick glance and anything that looks even a little funny will get tossed.  

I'm going to start with Kraken because honestly I feel like I understand how to use it better. (When I look at Blobplots, I find them interesting, but ultimately I am unsure what to do with them. So.) The thing with Kraken results is that it will often categorize a sequence as "human" (or something else) with almost no evidence. The sequence will be 2700 bases long, and it will find two kmers that look human, so it classifies the whole thing as a human sequence, ever though all the other ones were unclassified. So I need to stop it from doing this (because we kind of *want* things to be unclassified, at least if that means it's not a human or bacterium or something. But I obviously don't want it to be totally insensitive either, because that's the whole point. I'll need to write a parsing script for the output of kraken (this is the log file; there are tons of outputs) that can eventually filter these results. First though, I'll just have it spit out some stats so that I can figure out what kind of threshold might make sense. First first though, I need to run it. Hoping to do it with Snakemake, but we'll see.  

# Bacterial Datasets

I haven't been as focused on these, though maybe I should have been. This was a difficult process. I started by trying to avoid NCBI, because when has it ever been the one that worked the best? So I looked into Ensembl, which has loads and loads of options to download all kinds of things that are not genomes. And when I tried to download genomes, something about the accession I was trying to use didn't work.  

So then I tried the ENA, thinking that if that was where they were deposited first, maybe that would be easier. Alas, no. I got as far as downloading a program that wouldn't work without another piece of software that I wasn't allowed to download without an account. So I did all that, and it still wasn't working the way I needed it to. At this point, days had gone by. DAYS. So I dropped into the Hydra/Bioinformatics help session that day, and explained that this was just not working. Rebecca was great and in less than 20 minutes had a more promising solution (using NCBI, no less!) than I had been able to find.  

The key is this document: https://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/assembly_summary.txt, which has all of the bacterial genomes available on NCBI, and a good amount of info about them, including their locations (sort of). You can tell what info it has in more detail with this document: https://ftp.ncbi.nlm.nih.gov/genomes/README_assembly_summary.txt. It also helpfully has the fields numbered.  

I already had half of a script written to get a random sample of genomes (from when I was working with an Ensembl spreadsheet earlier in the week), so I just tweaked the first part to work with this new spreadsheet, and did a random sample of all the genomes. I tried downloading a couple manually, and the location on the spreadsheet downloads a file that has all the names of all the different types of data you can download, so I put together all the relevant bits of information and reconstructed the location of the full genome fasta file. All of this is here: /Users/jenniferlspillane/Desktop/Analyses/contamination/subsample_list.py, along with the spreadsheet of all the genomes.  

### Testing  

So, I did a little test to see if the script was working well (I mean, I did TINY tests first, but this was the first real one) I made it so that you can just select the number of genomes you want it to randomly sample, so it's really easy to change. I picked 117 to start (because there are 11,778,938 genomes in this file, and that seemed like eliminating enough decimal places off the end? IDK.) and generated a text file that I transferred onto Hydra. Then it was really easy to make a job that just had a wget command in it (no need for tons of memory or time) with the "-i" flag (to tell it to look in the file) and the file of the genome locations. It finished wicked fast (maybe two minutes? Maybe less) and downloaded 111 genome files.  

I know what you're thinking. "111? Didn't you tell me two seconds ago that you pulled out 117?" Yes, dear reader, I did. I'm not 100% sure what happened with those last six. The wget command (sort of annoyingly) spits out *lots* of stuff into the log file, so I just ended up grepping for the ones that went wrong like this `grep "ERROR 404" -B 3 test_genomes.log > did_not_work.txt` and saving them in that file. It's weird that the address at which it tried to find the genomes doesn't look complete in five of those examples, and it makes me wonder what went wrong, and why the script isn't putting "genomic.fna.gz" after the assembly name like it's supposed to. In one of the cases, the genome is names something super weird, with a lot of legit English words in it, so it's not really that surprising to me that that location ended up not working out. I'm not really sure if this is a problem or not yet. I'm going to have to download more and see how widespread it is. But also, I'm not sure I care? I don't really mind which genomes I have, as long as I have enough of them. That said, I would prefer that they were being left out somewhat randomly, and not systematically, since that might skew something.  

**Small Update:** The one that was really weird looking had spaces in the genome name, so obviously that wasn't going to work.  